# tensorboard

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

# float값으로 바꾸기 위해
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

In [3]:
# summary 값을 logs폴더에 저장
log_path = "./logs/xor"
#os.makedirs(log_path, exist_ok=True)
writer = tf.summary.create_file_writer(log_path) 
#코드에서 직접 TensorBoard 콜백을 설정하지 않아도 괜찮

In [4]:
#w1 = tf.Variable(tf.random.normal([2,1]), name='weight1')
#b1 = tf.Variable(tf.random.normal([1], name = 'bias1'))

#w2 = tf.Variable(tf.random.normal([2,1]), name='weight2')
#b2 = tf.Variable(tf.random.normal([1], name = 'bias2'))

#w3 = tf.Variable(tf.random.normal([2,1]), name='weight3')
#b3 = tf.Variable(tf.random.normal([1], name = 'bias3'))

#w4 = tf.Variable(tf.random.normal([2,1]), name='weight4')
#b4 = tf.Variable(tf.random.normal([1], name = 'bias4'))

w1 = tf.Variable(tf.random.normal((2, 10)), name='weight1')
b1 = tf.Variable(tf.random.normal((10,)), name='bias1')

w2 = tf.Variable(tf.random.normal((10, 10)), name='weight2')
b2 = tf.Variable(tf.random.normal((10,)), name='bias2')

w3 = tf.Variable(tf.random.normal((10, 10)), name='weight3')
b3 = tf.Variable(tf.random.normal((10,)), name='bias3')

w4 = tf.Variable(tf.random.normal((10, 1)), name='weight4')
b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

def neural_net(features, step):
    layer1 = tf.sigmoid(tf.matmul(features, w1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, w3) + b3)
    hypothesis = tf.sigmoid(tf.matmul(layer3, w4) + b4)
    with writer.as_default():
        tf.summary.histogram("weight1", w1, step=step)
        tf.summary.histogram("bias1", b1, step=step)
        tf.summary.histogram("layer1", layer1, step=step)

        tf.summary.histogram("weight2", w2, step=step)
        tf.summary.histogram("bias2", b2, step=step)
        tf.summary.histogram("layer2", layer2, step=step)

        tf.summary.histogram("weight3", w3, step=step)
        tf.summary.histogram("bias3", b3, step=step)
        tf.summary.histogram("layer3", layer3, step=step)

        tf.summary.histogram("weight4", w4, step=step)
        tf.summary.histogram("bias4", b4, step=step)
        tf.summary.histogram("hypothesis", hypothesis, step=step)

    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1-labels) * tf.math.log(1 - hypothesis))
    with writer.as_default():
        tf.summary.scalar('loss', cost, step=step)
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1) #0.01로 하니까 test 성능 0.5 ㄷㄷ ;;;

def accuracy_fn(hypothesis,labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(features, labels,step):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features,step), labels)
    return tape.gradient(loss_value, [w1,w2,w3,w4,b1,b2,b3,b4])

In [5]:
EPOCHS = 3000
for step in range(EPOCHS):
    for features, labels in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(features, labels, step)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [w1,w2,w3,w4,b1,b2,b3,b4]))
        if step % 50 == 0:
            loss_value = loss_fn(neural_net(features,step),labels)
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))

x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data,step), y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 0.7454
Iter: 50, Loss: 0.6929
Iter: 100, Loss: 0.6927
Iter: 150, Loss: 0.6927
Iter: 200, Loss: 0.6925
Iter: 250, Loss: 0.6923
Iter: 300, Loss: 0.6922
Iter: 350, Loss: 0.6921
Iter: 400, Loss: 0.6920
Iter: 450, Loss: 0.6917
Iter: 500, Loss: 0.6917
Iter: 550, Loss: 0.6915
Iter: 600, Loss: 0.6913
Iter: 650, Loss: 0.6911
Iter: 700, Loss: 0.6909
Iter: 750, Loss: 0.6906
Iter: 800, Loss: 0.6905
Iter: 850, Loss: 0.6901
Iter: 900, Loss: 0.6899
Iter: 950, Loss: 0.6896
Iter: 1000, Loss: 0.6892
Iter: 1050, Loss: 0.6888
Iter: 1100, Loss: 0.6884
Iter: 1150, Loss: 0.6877
Iter: 1200, Loss: 0.6872
Iter: 1250, Loss: 0.6866
Iter: 1300, Loss: 0.6858
Iter: 1350, Loss: 0.6849
Iter: 1400, Loss: 0.6838
Iter: 1450, Loss: 0.6827
Iter: 1500, Loss: 0.6810
Iter: 1550, Loss: 0.6796
Iter: 1600, Loss: 0.6774
Iter: 1650, Loss: 0.6749
Iter: 1700, Loss: 0.6718
Iter: 1750, Loss: 0.6682
Iter: 1800, Loss: 0.6638
Iter: 1850, Loss: 0.6584
Iter: 1900, Loss: 0.6524
Iter: 1950, Loss: 0.6456
Iter: 2000, Loss: 0.637

In [6]:
# terminal에서 cd DNN 하고 tensorboard --logdir=./logs/xor 하니 된다!!